# ASTRA Toolbox Trial Run

## Installation
In the terminal, run `conda env create --name xct --clone base` to create a new conda environment called `xct` based on the `base` environment. Then, run `conda activate xct` to activate the new environment.

Run `conda install -c astra-toolbox -c nvidia astra-toolbox` to install the ASTRA toolbox.

If any other packages are missing, such as ipykernel or numpy, run `conda install <package name>` to install them.

In [ ]:
# import astra
# import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import numpy as np
from os import mkdir
from os.path import join, isdir
from imageio import imread, imwrite
 
import astra

In [ ]:
astra.test()
astra.data3d.clear()
astra.projector.clear()
# gc.collect()

In [ ]:
""" 
Amy's Brain Dump Notes

3D Geometries

3D Data Objects

initializer will be the tif files merged into a single np array?

Part 1 of the tutorial is not necessary for this project, as it just synthetically creates the projections that we already have
We will follow Part 2 of the tutorial 
https://tomroelandts.com/articles/astra-toolbox-tutorial-reconstruction-from-projection-images-part-2

Longterm goal: 
Fully automated reconstruction of all 3000 projections of any sample

Short-term goal: 
Get a version working using test data, just to get the basics of ASTRA working

Order of business: 
1. Get a representation of the data (TIFF files) in the notebook
   - 3D np array? 
   - If size becomes an issue, try to take a subset of the data, i.e., the middle 30% of the data
2. Do math to get function parameter values
3. Continue following tutorial for the rest of the steps (e.g., sinograms, reconsruction, etc.)
4. Try running with projections

Major issue: 
Data is too big to fit into memory
Don't know why the kernel is crashing when trying to create projection geometry
   - Even when using a very small subset of the data (10 projections)...

Notes: 
- ASTRA expects the cumulative projections to be in an array of shape (detector_rows, num_projections, detector_cols)
   - i.e., 2316 x 3001 x 2316
- cols = detector_cols, rows = num_projections, slices = detector_rows

"""

In [ ]:
im = Image.open('Projections/Mitos_Phantom/Projection_1.tif')
imarray = np.array(im)
print(imarray.dtype)
print(imarray)
plt.imshow(im, cmap='gray', vmin=0, vmax=65535)
plt.show()


In [ ]:
distance_source_origin = 485  # [mm]
distance_origin_detector = 285  # [mm]
detector_pixel_size = 0.05  # [mm]
detector_rows = 2316  # Vertical size of detector [pixels].
detector_cols = 2316  # Horizontal size of detector [pixels].
num_of_projections = 3001
""" MUST REMOVE THE [:10] AFTER TESTING"""
angles = np.linspace(0, 2 * np.pi, num=num_of_projections, endpoint=False)[:10]

In [ ]:
# Read projections into a 3D numpy array

""" THIS LINE IS TEMPORARY!! Just for testing purposes"""
num_of_projections = 10

input_dir = 'Projections/Mitos_Phantom/'
projections = np.zeros((detector_rows, num_of_projections, detector_cols))

for i in range(1, num_of_projections+1): 
  im = plt.imread("{}Projection_{}.tif".format(input_dir, i))
  # im /= 65535
  projections[:, i-1, :] = im
  # print("{}Projection_{}.tif".format(input_dir, i))



In [ ]:
# Translate projection images into ASTRA Toolbox format

# Define the projection geometry
proj_geom = astra.create_proj_geom('cone',  1, 1, detector_rows, detector_cols, angles, (distance_source_origin + distance_origin_detector) / detector_pixel_size, 0)

# Create sinograms
projections_id = astra.data3d.create('-sino', proj_geom, projections)

In [ ]:
# Create reconstruction

# Define geometry
vol_geom = astra.create_vol_geom(detector_cols, detector_cols, detector_rows)
print(vol_geom)

In [ ]:
# Reconstruction algorithm
""" Keeps crashing right here... """
try: 
  reconstruction_id = astra.data3d.create('-vol', vol_geom, data=0)
except Exception as e: 
  print(f"Failed: {e}")

alg_cfg = astra.astra_dict('FDK_CUDA')
alg_cfg['ProjectionDataId'] = projections_id
alg_cfg['ReconstructionDataId'] = reconstruction_id
algorithm_id = astra.algorithm.create(alg_cfg)
astra.algorithm.run(algorithm_id)

reconstruction = astra.data3d.get(reconstruction_id)


In [ ]:
# Save reconstructed slices

# Remove any negative values
reconstruction[reconstruction < 0] = 0
# Normalize
reconstruction /= np.max(reconstruction)
reconstruction = np.round(reconstruction * 255).astype(np.uint8)

# Save images
output_dir = 'Reconstructions/Mitos_Phantom/'

for i in range(detector_rows):
    im = reconstruction[i, :, :]
    im = np.flipud(im)
    imwrite(join(output_dir, 'reco%04d.png' % i), im)